# Linear Algebra using Python's `numpy` library



The purpose of this notebook is to develop a basic familiarity with Python's native linear algebra support.
This is accomplished in two parts, the first being motivated by different types of matrix factorization and
the second motivated by graph theory.



* Part 1: Factoring matrices in six different ways (courtesy Gilbert Strang)
* Part 2: Spectral graph theory, adapting a graph to matrix representation and then 'reading the eigenvalue tea leaves'



> Is this notebook fully realized? No! Not by a long shot! It is under development. Here are some issues.
> - reference versus value passing should be doped out: Make sure not breaking values
> - same with mutable / immutable: should be clear
> - use NetworkX representation of G?


## Part 1: Matrix factorizations


[According to our dear friend Gilbert Strang](https://youtu.be/YrHlHbtiSM0?si=MXanoTEHWOVHNUFX) 
there are five essential factorization plans for a given matrix $A$ as the basis, pardon
the double entendre, for beginning linear algebra. Here the matrix $A$ consists of a rectangular 
array of numbers that corresponds to a linear transform operating on a vector space.


Before listing the factorizations I will mention the above link is a five minute overview.
The full on 'crash course hour-long lecture' on this topic by Professor Strang is 
[here](https://youtu.be/nTwRjQ4xqUc?si=k0Dui4Y_lC737F2y), highly recommended.


Here are those five factorizations as a list of six. Because let's face it, 
six is equal to five for small values of six.


1) $A = C \cdot R$ where $rank(A) = r$ gets us to the $C$-matrix column space and the $R$-matrix row space, respectively $(m \times r) \; (r \times n)$


2) $A = LU$ where $A$ is typically square and $L$ and $U$ are respectively *lower* and *upper* triangular... sometimes a permutation matrix $P$ is introduced as well ($PA=LU$)


3) $A = Q \cdot R$ where the columns of $Q$ are orthogonal vectors


4) $S = Q \Lambda Q^T$ giving an eigendecomposition from $S q = \lambda q$


5) $A = X \Lambda X^{-1}$ more eigen-development, $A x = \lambda x$


6) $A = U \Sigma V^T$ for Singular Value Decomposition


I will intersperse more extensive comments in the subsections that follow.

### Part 1.0 Building a matrix A


Using the Python `numpy` and `scipy` libraries (abbreviated `np` and `sp`) we have a 
drawer full of functions and attributes on hand. This includes an implicit 
format for matrix content based on the multidimensional array structure
called `ndarray` in `numpy`. 


This section builds a test matrix $A$ which is subjected to factorization
in subsequent sections. 


- Below is a non-trivial example of the construction syntax (from a list of lists). 
- There is a bias towards row-format...
    - ...but the array has an *attribute* **`.T`**: The transpose of the ndarray
- Another common attribute is **`.shape`**: The two dimensions (rows and columns) of the ndarray 


```
import numpy as np

row0 = [1, 2]
row1 = [-1, 1]
A = np.array([row0, row1])
print(A, '\n\n', A.T)

[[ 1  2]
 [-1  1]] 

 [[ 1 -1]
 [ 2  1]]
```

In [1]:
import numpy as np, scipy as sp
from random import random, randint    # randint range is inclusive


def generate_random_integer_valued_matrix(n, m, low_limit, high_limit):
    matrix_list = []
    for row_counter in range(n_rows):
        matrix_list.append([randint(low_limit, high_limit) for i in range(m_cols)])
    return np.array(matrix_list)


In [2]:
n_rows, m_cols = 5,5                # experiment with these!

A = generate_random_integer_valued_matrix(n_rows, m_cols, -3, 3)

A

array([[-1, -2, -1, -2, -2],
       [-1,  0, -1,  0,  2],
       [ 1,  0, -1, -1,  1],
       [ 0,  2, -1,  2, -2],
       [ 2,  3,  1,  0,  2]])

`A` is our matrix of interest; and technically in Python it is a **numpy.ndarray** with dimension 2. 


Again: `A.shape` is an *attribute* of the `A` object. In fact it is
a 2-tuple with values `(n_rows, m_cols)`. These values can be used directly as `A.shape[0]` and `A.shape[1]`.


An element of `A` can be indexed using a tuple-subscript or a multi-subscript: `A[0, 2]` or `A[0][2]`.
The first value is row index, then column index.


Often matrix operations produce goofy scientific notation results like `0.999999e+00` instead of `1`.
I use the `round(a, b)` function to clean this up and make it more readable. 

In [3]:
if not A.shape == (n_rows, m_cols): print("A.shape is bent outta shape!")       # sanity check
print('Check this indexing against the printout of A above: A[0,2] versus A[2][0]: ', A[0,2], ' versus ', A[2][0])

Check this indexing against the printout of A above: A[0,2] versus A[2][0]:  -1  versus  1


### Factorization 1: A = C R

### Factorization 2: A = L U


This is the bread and butter of solving $A x = b$ for unknown $x$. The idea is to produce lower- and upper-triangular matrices
$L$ and $U$. Now we have $A x = b$ with $x$ what we are solving for. Substitute:


$L U x = b$


Since matrix multiplication is associative, $(AB)C = A(BC)$, we can write this as


$L (U x) = b$


Suppose $U x$ gives us a vector $c$: $U x = c$. Then $L c = b$. Since $L$ is lower-triangular it is quite easy to solve for this $c$ vector. 
Then we can proceed to solve for $x$ in $U x = c$ (because $c$ is now known and again 'triangular makes solving easy); and there we are at the solution. 

In [5]:
np.set_printoptions(suppress=True)        # better-looking matrix prints

print('matrix A:\n\n', A, '\n\n')

P, L, U = sp.linalg.lu(A)

print('permutation matrix P:\n\n', P, '\n\n')
print('L:\n\n', np.around(L, 3), '\n\n')
print('U:\n\n', np.around(U, 3), '\n\n')

PLU_recovered = np.matmul(P, np.matmul(L, U))

print("PLU - A: \n\n", PLU_recovered - A, '\n\n')

matrix A:

 [[-1 -2 -1 -2 -2]
 [-1  0 -1  0  2]
 [ 1  0 -1 -1  1]
 [ 0  2 -1  2 -2]
 [ 2  3  1  0  2]] 


permutation matrix P:

 [[0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]] 


L:

 [[ 1.     0.     0.     0.     0.   ]
 [ 0.     1.     0.     0.     0.   ]
 [ 0.5   -0.75   1.     0.     0.   ]
 [-0.5   -0.25   0.333  1.     0.   ]
 [-0.5    0.75  -0.111  0.867  1.   ]] 


U:

 [[ 2.     3.     1.     0.     2.   ]
 [ 0.     2.    -1.     2.    -2.   ]
 [ 0.     0.    -2.25   0.5   -1.5  ]
 [ 0.     0.     0.    -1.667 -1.   ]
 [ 0.     0.     0.     0.     5.2  ]] 


PLU - A: 

 [[ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0. -0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]] 




### Factorization 3: A = Q R

In [6]:
q_factor, r_factor = np.linalg.qr(A)
A_recovered = np.matmul(q_factor, r_factor)
print(A, '\n\n')
print(q_factor, '\n\n')
print(r_factor, '\n\n')
print(A_recovered, '\n\n')

[[-1 -2 -1 -2 -2]
 [-1  0 -1  0  2]
 [ 1  0 -1 -1  1]
 [ 0  2 -1  2 -2]
 [ 2  3  1  0  2]] 


[[-0.37796447  0.30578831 -0.32177297  0.58619035  0.56256502]
 [-0.37796447 -0.40771775 -0.2649895   0.44647658 -0.64911348]
 [ 0.37796447  0.40771775 -0.77604068 -0.14578827 -0.25964539]
 [ 0.         -0.71350607 -0.46373163 -0.29765106  0.43274232]
 [ 0.75592895 -0.2548236   0.09463911  0.5892276   0.08654846]] 


[[ 2.64575131  3.02371578  1.13389342  0.37796447  1.88982237]
 [ 0.         -2.80305955  0.15289416 -2.44630652 -0.10192944]
 [ 0.          0.          1.92117388  0.49212336  0.45426772]
 [ 0.          0.          0.         -1.62189453  1.34854152]
 [ 0.          0.          0.          0.         -3.37539011]] 


[[-1. -2. -1. -2. -2.]
 [-1. -0. -1. -0.  2.]
 [ 1.  0. -1. -1.  1.]
 [ 0.  2. -1.  2. -2.]
 [ 2.  3.  1.  0.  2.]] 




### Factorization 4 / 5

### Factorization 6 SVD

In [7]:
print(A, '\n\n')
U, Sigma, VT = np.linalg.svd(A)
print(U, '\n\n')
print(np.diag(Sigma), '\n\n')
print(VT, '\n\n')
A_recovered = np.matmul(U, np.matmul(np.diag(Sigma), VT))
print(A_recovered, '\n\n')

[[-1 -2 -1 -2 -2]
 [-1  0 -1  0  2]
 [ 1  0 -1 -1  1]
 [ 0  2 -1  2 -2]
 [ 2  3  1  0  2]] 


[[-0.64525455 -0.06332216  0.32988711  0.44603025 -0.52140999]
 [ 0.07518025 -0.34737698 -0.82633412  0.26158908 -0.34988604]
 [ 0.07692647 -0.32580057  0.1067238   0.70554142  0.61543289]
 [ 0.13198976  0.86584469 -0.2036708   0.43686486 -0.02364278]
 [ 0.74475198 -0.13959408  0.39430274  0.20973454 -0.47580957]] 


[[5.46363179 0.         0.         0.         0.        ]
 [0.         3.92308798 0.         0.         0.        ]
 [0.         0.         2.30727807 0.         0.        ]
 [0.         0.         0.         1.94313069 0.        ]
 [0.         0.         0.         0.         0.81167689]] 


[[ 0.39104113  0.69344801  0.20241299  0.2704359   0.50210567]
 [-0.04952466  0.36694345 -0.0685529   0.55673854 -0.740434  ]
 [ 0.60321133  0.05018571  0.42807876 -0.50875516 -0.43764625]
 [ 0.21480345  0.31437558 -0.84414861 -0.37252883 -0.0605217 ]
 [ 0.65926452 -0.53210126 -0.2418495   0.

## Part 2: Laplacian eigenvalues from spectral graph theory



Here the idea is to create matrices from graphs, of a particular form named after Laplace.
Once we have such a matrix the resulting eigenvalues are associated with characteristics 
of the graph. Kind of magical! My reference is **Spectral Graph Theory** by Fan Chung.


Incidentally the ideal way to proceed would be from the 'two set' idea for forming a graph.
The first set $V$ has some number of unique elements and the second set $E$ consists of pairs
of elements of $V$. Now Python has `set` as a *type* so at first glance maybe that is the way to go.

In [8]:
# Let's start by constructing random graph G
# The vertices n in number are simply identified by integers 0 ... n-1.
# Two vertices are connected by an edge with probability p.
# Setting p to 1/2 pretty much guarantees G is connected
# Self-loops are not allowed.
# A set in Python is immutable (I think)


n = 7
p = 0.5
V = set(range(n))


E = []
for i in range(n-1):
    for j in range(i+1, n):
        if random() < p:
            E.append((i, j))

E = set(E)
e = len(E)

print(V, '\n\n')
print(E, '\n\n')
print('There are ' + str(n) + ' vertices and ' + str(e) + ' edges.\n')


{0, 1, 2, 3, 4, 5, 6} 


{(0, 1), (3, 4), (1, 4), (0, 5), (3, 6), (1, 6)} 


There are 7 vertices and 6 edges.



In [9]:
import networkx as nx

### Section 2.0 Build a graph G


Analogous to Section 1 where we build matrix A to be factored, here we want to build a graph G in the NetworkX library context. 


G = 1               # maybe upgrade this to a graph